In [15]:
#Importing and loading libraries
import numpy as np
import pandas as pd
#For oversampling
from collections import Counter
from imblearn.over_sampling import ADASYN
#Train-test sets splitting
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
#For DecisionTrees
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
#For Logistic Regression classifier
from sklearn.linear_model import LogisticRegression
#For Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
#For Support Vector Machine
from sklearn import svm
#For Random Forest
from sklearn.ensemble import RandomForestClassifier

In [16]:
df = pd.read_csv("C:\\Users\\akram\\Desktop\\Dissertation\\Datasets\\PCDataset.csv")

features = df.drop(['InjuredNextTrainingSession'], axis = 1)
target = df['InjuredNextTrainingSession']

In [24]:
#Split dataset into 70% training set, 30% test set; random state of 1 ensures
#same distribution every time
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, random_state = 1)
#Looking at proportion of '0's and '1's in the target column
imbal_proportion = Counter(y_train)
print(imbal_proportion)
#Oversampling the training set
oversample = ADASYN()
x_train, y_train = oversample.fit_resample(x_train, y_train)
#Looking at new proportion of '0's and '1's
bal_proportion = Counter(y_train)
print(bal_proportion)

Counter({0: 2661, 1: 29})
Counter({1: 2668, 0: 2661})


In [17]:
#Function to return accuracy metrics
def accuraciz(y_test, y_pred):
    count = Counter(y_test)
    real_values_zero = count[0]
    real_values_one = count[1]
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    #Transform the data into a numerical vector
    y_test = list(map(int, y_test))
    y_pred = list(map(int, y_pred))
    
    for i in range(len(y_pred)):
        if y_test[i] == 1 and y_pred[i] == 1:
            TP += 1
        if y_pred[i] == 1 and y_test[i] != y_pred[i]:
            FP += 1
        if y_test[i] == y_pred[i] == 0:
            TN += 1
        if y_pred[i] == 0 and y_test[i] != y_pred[i]:
            FN += 1
    
    sensitivity = TP/(TP + FN)
    specificity = TN/(FP + TN)
    FPR = FP / real_values_zero
    FNR = FN / real_values_one
    acc = (TP + TN) / (real_values_one + real_values_zero)
    ba = (sensitivity + specificity) / 2
    f1 = (2*TP) / (2*TP + FP + FN)
    LRplus = sensitivity/(1-specificity)
    LRminus = (1-sensitivity)/specificity
    
    return sensitivity*100, specificity*100, FPR*100, FNR*100, acc*100, ba*100, f1*100, LRplus, LRminus, TP, FP, TN, FN

In [25]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myDT = DecisionTreeClassifier(criterion = 'gini', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myDT = cross_val_predict(myDT, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myDT)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 322324 (42.38316896778435%), FP: 78834 (10.366074950690335%)
TN: 301666 (39.66679815910585%), FN: 57676 (7.583957922419461%)
Sensitivity has mean 84.82210526315787 with variance 1.653935180055402
Specificity has mean 79.2814717477004 with variance 0.8156115215991142
False Positive Rate has mean 20.718528252299606 with variance 0.8156115215991128
False Negative Rate has mean 15.177894736842106 with variance 1.6539351800554019
ACC has mean 82.04996712689021 with variance 0.6822235795077547
Balanced Accuracy has mean 82.05178850542916 with variance 0.6824989238402156
F1-Score has mean 82.52176233657954 with variance 0.7363162326447827
Likelihood ratio positive has mean 4.102182727761354 with variance 0.03891381917665675
Likelihood ratio negative has mean 0.19148862877208053 with variance 0.0002777822943218712


In [26]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myDT = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myDT = cross_val_predict(myDT, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myDT)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 322253 (42.37383300460223%), FP: 85528 (11.246285338593031%)
TN: 294972 (38.78658777120316%), FN: 57747 (7.593293885601578%)
Sensitivity has mean 84.80342105263158 with variance 2.5810035318559583
Specificity has mean 77.5222076215506 with variance 1.7801057810025904
False Positive Rate has mean 22.477792378449408 with variance 1.7801057810025884
False Negative Rate has mean 15.19657894736842 with variance 2.581003531855956
ACC has mean 81.16042077580539 with variance 0.8868222703755987
Balanced Accuracy has mean 81.16281433709109 with variance 0.8870849912618727
F1-Score has mean 81.80879385287088 with variance 0.9386740783761822
Likelihood ratio positive has mean 3.7853050053987816 with variance 0.05068241592961184
Likelihood ratio negative has mean 0.19601652153861518 with variance 0.0004166378982258184


In [27]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Logistic Regression classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myCLR = LogisticRegression(solver='lbfgs', max_iter=1000)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myCLR = cross_val_predict(myCLR, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myCLR)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 228517 (30.048257725180804%), FP: 194479 (25.572518080210386%)
TN: 186021 (24.4603550295858%), FN: 151483 (19.91886916502301%)
Sensitivity has mean 60.13605263157895 with variance 6.661288850415514
Specificity has mean 48.88856767411301 with variance 0.6474266345029807
False Positive Rate has mean 51.11143232588699 with variance 0.6474266345029801
False Negative Rate has mean 39.86394736842105 with variance 6.661288850415514
ACC has mean 54.50861275476658 with variance 1.6715802304878296
Balanced Accuracy has mean 54.51231015284598 with variance 1.6735563178026902
F1-Score has mean 56.8949194347716 with variance 3.0147495364726513
Likelihood ratio positive has mean 1.1767441499551787 with variance 0.0026036652170363488
Likelihood ratio negative has mean 0.8154957284370871 with variance 0.0027782572850544467


In [28]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Naive-Bayes classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myNB = GaussianNB()

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myNB = cross_val_predict(myNB, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myNB)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 299916 (39.43668639053254%), FP: 254482 (33.46245890861275%)
TN: 126018 (16.570414201183432%), FN: 80084 (10.530440499671268%)
Sensitivity has mean 78.92526315789475 with variance 0.10872188365651021
Specificity has mean 33.11905387647832 with variance 0.06491741794892619
False Positive Rate has mean 66.88094612352168 with variance 0.06491741794892616
False Negative Rate has mean 21.074736842105267 with variance 0.1087218836565097
ACC has mean 56.00710059171597 with variance 0.06251543047780368
Balanced Accuracy has mean 56.02215851718653 with variance 0.062529819868473
F1-Score has mean 64.19435697372705 with variance 0.04865492068579222
Likelihood ratio positive has mean 1.180111647725781 with variance 6.485853606586872e-05
Likelihood ratio negative has mean 0.6364049719777156 with variance 0.0001668873439256709


In [29]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
mySVC = svm.SVC()

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_mySVC = cross_val_predict(mySVC, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_mySVC)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 300462 (39.508481262327415%), FP: 240515 (31.625904010519395%)
TN: 139985 (18.406969099276793%), FN: 79538 (10.458645627876397%)
Sensitivity has mean 79.06894736842105 with variance 0.47735152354570604
Specificity has mean 36.78975032851511 with variance 0.06674080200856126
False Positive Rate has mean 63.21024967148489 with variance 0.06674080200856145
False Negative Rate has mean 20.93105263157895 with variance 0.47735152354570637
ACC has mean 57.91545036160421 with variance 0.11368850124122483
Balanced Accuracy has mean 57.929348848468074 with variance 0.11382341336173234
F1-Score has mean 65.24745980428679 with variance 0.13948407307943397
Likelihood ratio positive has mean 1.2508978581870003 with variance 0.00011804880721910943
Likelihood ratio negative has mean 0.5689323284780458 with variance 0.0003301913349506424


In [30]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myRF = RandomForestClassifier(criterion = 'entropy', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myRF = cross_val_predict(myRF, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myRF)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 333544 (43.85851413543721%), FP: 56655 (7.449704142011834%)
TN: 323845 (42.58316896778435%), FN: 46456 (6.108612754766601%)
Sensitivity has mean 87.77473684210527 with variance 0.5455307479224374
Specificity has mean 85.11038107752957 with variance 0.22213492517107788
False Positive Rate has mean 14.889618922470433 with variance 0.2221349251710782
False Negative Rate has mean 12.225263157894737 with variance 0.5455307479224374
ACC has mean 86.44168310322155 with variance 0.16745582013978377
Balanced Accuracy has mean 86.44255895981743 with variance 0.1675620369192049
F1-Score has mean 86.61135323208622 with variance 0.18797352276755447
Likelihood ratio positive has mean 5.9007012121575295 with variance 0.03464679752898727
Likelihood ratio negative has mean 0.14363774767588144 with variance 7.400136836044761e-05


In [14]:
#Random Baseline
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

i = 1
while (i < 101) :
    #All zero vector
    b2 = np.random.randint(2, size = len(target))

    performance = accuraciz(target, b2)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    i += 1
    

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 1914 (0.49804839968774395%), FP: 190823 (49.654696851418166%)
TN: 189677 (49.35649232370544%), FN: 1886 (0.4907624251886547%)
Sensitivity has mean 50.36842105263158 with variance 48.340720221606674
Specificity has mean 49.849408672798944 with variance 0.642585642724059
False Positive Rate has mean 50.150591327201056 with variance 0.6425856427240594
False Negative Rate has mean 49.63157894736841 with variance 48.34072022160668
ACC has mean 49.854540723393185 with variance 0.6447996021843302
Balanced Accuracy has mean 50.10891486271527 with variance 12.504564968053769
F1-Score has mean 1.948212684823741 with variance 0.07314726007286222
Likelihood ratio positive has mean 1.0048036830124625 with variance 0.019842643101759195
Likelihood ratio negative has mean 0.9960986982247273 with variance 0.020197975888147427
